# End to End Pure Streaming Data-Pipeline for Building Maintenance Table Using Spark Structured Streaming on Databricks

###### Description: In this notebook we read building_maintenance state rows from incoming csv files into a streamig dataframe, transform (clean, cast, rename) the data, add/update the latest state to a Databricks Delta table
###### Objective: (incoming csv files) --> "building_maintenance_streamingDF" --> "results_df" --> "building_maintenance_data"

In [3]:
import requests
import json
import optimus as op
import phonenumbers 
import re
import datetime
import time

from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col

In [4]:
# Schema for Building Maintenance
building_maintenance_schema = StructType([
            StructField("Maintenance_id", IntegerType(), False),
            StructField("Building_name", StringType(), True),
            StructField("Ndate", TimestampType(), False),
            StructField("Issue_reported", StringType(), True),
            StructField("Contractor_id", IntegerType(), True), 
            StructField("Resolution", StringType(), True), 
            StructField("Status", StringType(), True),
            StructField("event_time", TimestampType(), True)])

building_maintenance_udf_schema = StructType([
            StructField("Building_name", StringType(), True),
            StructField("Ndate", TimestampType(), False),
            StructField("Issue_reported", StringType(), True),
            StructField("Contractor_id", IntegerType(), True), 
            StructField("Resolution", StringType(), True), 
            StructField("Status", StringType(), True),
            StructField("event_time", TimestampType(), True)])

###### Description: Get building_maintenance csv files as a streaming "building_maintenance_streamingDF" and process it on the fly and get transformed stream "building_maintenance_df"
###### Objective: (incoming csv files) --> "building_maintenance_streamingDF" --> "building_maintenance_df"

In [6]:
# Get building_maintenance Steaming DataFrame from csv files

# streaming starts here by reading the input files 
building_maintenance_Path = "/FileStore/apartment/building_maintenance/inprogress/"
building_maintenance_streamingDF = (
  spark
    .readStream
    .schema(building_maintenance_schema)
    .option("maxFilesPerTrigger", "1")
    .option("header", "true")
    .option("multiLine", "true")
    .csv(building_maintenance_Path)
)
# Clear invalid rows
building_maintenance_df = building_maintenance_streamingDF.select("*").where("Maintenance_id IS NOT NULL")
# Instantiation of DataTransformer class:
transformer = op.DataFrameTransformer(building_maintenance_df)
# Replace NA with 0's
transformer.replace_na(0.0, columns="*")
# Clear accents: clear_accents only from name column and not everywhere 
transformer.clear_accents(columns='*')
# Remove special characters:  From all Columns 
# transformer.remove_special_chars(columns=['building_maintenance_name', 'Address_line_1', 'City', 'Post_code', 'Region'])

##### This function parses the corresponding columns into a single column

In [8]:
def my_fun(Building_name, Ndate, Issue_reported, Contractor_id, Resolution, Status, event_time):
  return zip(Building_name, Ndate, Issue_reported, Contractor_id, Resolution, Status, event_time)

udf_Fun = udf(my_fun, ArrayType(building_maintenance_udf_schema))

In [9]:
intermediate_df = ( building_maintenance_df.withWatermark("event_time", "10 seconds")
            .groupBy("Maintenance_id")
            .agg(F.collect_list("Building_name").alias("Building_name"),
                 F.collect_list("Ndate").alias("Ndate"),
                 F.collect_list("Issue_reported").alias("Issue_reported"),
                 F.collect_list("Contractor_id").alias("Contractor_id"), 
                 F.collect_list("Resolution").alias("Resolution"), 
                 F.collect_list("Status").alias("Status"), 
                 F.collect_list("event_time").alias("event_time"), 
                 F.max("event_time").alias("latest_event_time"))
            .select("Maintenance_id", 
                    F.explode(udf_Fun(F.column("Building_name"), 
                                      F.column("Ndate"), 
                                      F.column("Issue_reported"), 
                                      F.column("Contractor_id"), 
                                      F.column("Resolution"), 
                                      F.column("Status"), 
                                      F.column("event_time")))
                    .alias("data"), "latest_event_time"))

##### Filter the data where event_time is latest

In [11]:
results_df = (intermediate_df
              .select("Maintenance_id", 
                      "data.Building_name", 
                      "data.Ndate", 
                      "data.Issue_reported", 
                      "data.Contractor_id", 
                      "data.Resolution", 
                      "data.Status",  
                      "data.event_time", 
                      "latest_event_time")
              .where("data.event_time=latest_event_time")).orderBy("Maintenance_id")

##### Display final result
###### This result shows the latest state of all the unique building_maintenance_id

In [13]:
display(results_df)

##### Below cells are optional if external functionality or storage is needed

###### Write the stream to a Databricks Delta table for storage

In [16]:
streaming_query = (results_df.writeStream
 .format("delta")
 .outputMode("complete")
 .option("mergeSchema", "true")
 .option("checkpointLocation", "/delta/apartment/building_maintenance/_checkpoints/streaming-agg")
 .start("/delta/apartment/building_maintenance_data"))

#### Read the Delta Table as a Static or Streaming DataFrame
#### This dataframe will always be Up-To-Date

In [18]:
building_maintenance_data = spark.read.format("delta").load("/delta/apartment/building_maintenance_data").orderBy("Maintenance_id")

In [19]:
display(building_maintenance_data)

### Do Some Live Streaming Graphs

In [21]:
building_maintenance_data_stream = spark.readStream.format("delta").load("/delta/apartment/building_maintenance_data")

In [22]:
display(building_maintenance_data_stream.groupBy("Status").count())